# Preprocess corpus

In [ ]:
import pandas as pd

class ProcessCorpus:
    def __init__(self, filepath='/kaggle/input/bkai-ai-track2-legal-document-retrieval/Legal Document Retrieval/corpus.csv'):
        self.df = pd.read_csv(filepath, usecols=['text', 'cid'])

    def process_text(self):
        self.df['text'] = self.df['text'].apply(lambda x: ' '.join(x.split()))
        self.save_to_csv()

    def save_to_csv(self):
        output_path = 'preprocessed_corpus.csv'
        self.df.to_csv(output_path, index=False)


if __name__ == "__main__":
    processor = ProcessCorpus()
    processor.process_text()

# Encode 
- preprocessed_corpus.csv to encoded_corpus.json
- public_test.csv to encoded_public_test.json
- train.csv to encoded_train_full.json

In [2]:
import pandas as pd
import numpy as np
import torch
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Encode corpus.csv
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_csv('/kaggle/input/preprocessed-corpus/preprocessed_corpus.csv')

model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')
model = model.to(device)

def encode(lst = [], convert_to_tensor=True, batch_size=128):
    vectors = []
    with tqdm(total = len(lst), desc="Encoding texts") as pbar:
        # Process in batches
        for i in range(0, len(lst), batch_size):
            batch = lst[i:i + batch_size]
            encoded_batch = model.encode(batch, convert_to_tensor=True, device=device)
            # Move to CPU before converting to numpy
            if torch.cuda.is_available():
                encoded_batch = encoded_batch.cpu()
            vectors.extend([np.array(arr) for arr in encoded_batch.numpy()])
            pbar.update(len(batch))
    return vectors

df['vector'] = encode(lst=list(df['text']))

df.to_json('encoded_corpus.json')


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Encoding texts:   0%|          | 0/261597 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding texts:   0%|          | 128/261597 [00:01<44:54, 97.03it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding texts:  13%|█▎        | 33920/261597 [04:01<25:51, 146.78it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding texts:  13%|█▎        | 33920/261597 [04:01<27:00, 140.47it/s]


KeyboardInterrupt: 

In [3]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Encode public_test
df = pd.read_csv('/kaggle/input/bkai-ai-track2-legal-document-retrieval/Legal Document Retrieval/public_test.csv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder', device=device)

def encode(lst = [], convert_to_tensor=True, batch_size=128):
    vectors = []
    with tqdm(total=len(lst), desc="Encoding questions") as pbar:
        for i in range(0, len(lst), batch_size):
            batch = lst[i:i + batch_size]
            encoded_batch = model.encode(batch, convert_to_tensor=True)
            if device == 'cuda':
                encoded_batch = encoded_batch.cpu()
            vectors.extend([np.array(arr) for arr in encoded_batch.numpy()])
            pbar.update(len(batch))
    return vectors

df['question_vector'] = encode(lst=list(df['question']))

output_df = df[['question', 'qid', 'question_vector']]

output_df.to_json('encoded_public_test.json')

Encoding questions:   0%|          | 0/10000 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding questions:   1%|▏         | 128/10000 [00:00<00:15, 646.17it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding questions:   3%|▎         | 256/10000 [00:00<00:12, 757.95it/s]

...

Encoding questions:  99%|█████████▊| 9856/10000 [00:11<00:00, 890.36it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding questions: 100%|█████████▉| 9984/10000 [00:11<00:00, 906.29it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Encoding questions: 100%|██████████| 10000/10000 [00:11<00:00, 877.61it/s]


In [4]:
import pandas as pd
import numpy as np
import torch
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Encode train.csv
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_csv('/kaggle/input/bkai-ai-track2-legal-document-retrieval/Legal Document Retrieval/train.csv')

model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')
model = model.to(device)

def encode(lst = [], convert_to_tensor=True, batch_size=1024):
    vectors = []
    with tqdm(total=len(lst), desc="Encoding texts") as pbar:
        # Process in batches
        for i in range(0, len(lst), batch_size):
            batch = lst[i:i + batch_size]
            encoded_batch = model.encode(batch, convert_to_tensor=True, device=device)
            # Move to CPU before converting to numpy
            if torch.cuda.is_available():
                encoded_batch = encoded_batch.cpu()
            vectors.extend([np.array(arr) for arr in encoded_batch.numpy()])
            pbar.update(len(batch))
    return vectors

df['question_vector'] = encode(lst=list(df['question']))

df.to_json('encoded_train_full.json')


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Encoding texts:   0%|          | 0/119456 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Encoding texts:   1%|          | 1024/119456 [00:00<01:51, 1066.36it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

...

Encoding texts:  99%|█████████▊| 117760/119456 [01:49<00:01, 1074.29it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Encoding texts:  99%|█████████▉| 118784/119456 [01:50<00:00, 1071.94it/s]

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Encoding texts: 100%|██████████| 119456/119456 [01:51<00:00, 1072.47it/s]


# Create a dissimilarity list and negative pairing

In [1]:
import torch
import torch.nn.functional as F
import pandas as pd
import zipfile
import json

# Input: corpus and training dataset
# Output: dissimilarity list

train_df = pd.read_json('/kaggle/input/encodedtrainfull/encoded_train_full.json')
corpus_df = pd.read_json('/kaggle/input/encoded/encoded_corpus.json')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

train_ids = train_df['qid'].tolist()
train_vectors = torch.tensor(train_df['question_vector'].tolist(), dtype=torch.float32).to(device)
corpus_ids = corpus_df['cid'].tolist()
corpus_vectors = torch.tensor(corpus_df['vector'].tolist(), dtype=torch.float32).to(device)

train_vectors = F.normalize(train_vectors, p=2, dim=1)
corpus_vectors = F.normalize(corpus_vectors, p=2, dim=1)

json_results = []
batch_size = 256

with open('predict_most_dissimilar.txt', 'w') as f:
    for i in range(0, len(train_ids), batch_size): 
        # Process in batches
        batch_train_vectors = train_vectors[i:i+batch_size]
        batch_train_ids = train_ids[i:i+batch_size]
        
        similarity_matrix = torch.matmul(batch_train_vectors, corpus_vectors.T)
        
        min_values, min_indices = torch.min(similarity_matrix, dim=1)
        
        for j, (idx, score) in enumerate(zip(min_indices, min_values)):
            qid = batch_train_ids[j]
            cid = str(corpus_ids[idx.item()])
            
            f.write(f"{qid} {cid}\n")

            json_entry = {
                "query_id": qid,
                "candidates": {
                    "doc_ids": [cid],
                    "scores": [score.item()],
                }
            }
            json_results.append(json_entry)

with open('predict_most_dissimilar.json', 'w', encoding='utf-8') as f:
    json.dump(json_results, f, ensure_ascii=False, indent=2)

with zipfile.ZipFile('predict_most_dissimilar.zip', 'w') as zipf:
    zipf.write('predict_most_dissimilar.txt')
    zipf.write('predict_most_dissimilar.json')


Using device: cuda


In [2]:
import os
import subprocess
from IPython.display import FileLink, display

# Download the dissimilarity list to local

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

# download_file('/kaggle/working', 'encoded_public_test_finetuned')
download_file('/kaggle/working', 'predict_dissimilar')

/kaggle/working/predict_dissimilar.zip

In [3]:
import pandas as pd
import json
import numpy as np

#Input: 
''' 
        DataFrames: df_train, df_corpus - csv input
        json: predict_most_dissimilar.json - 
    '''
#Output
''' 
        json: fine_tune_training.json -> will be used to fine-tune the model
    '''



df_train = pd.read_csv('/kaggle/input/bkai-ai-track2-legal-document-retrieval/Legal Document Retrieval/train.csv', on_bad_lines='skip')
df_corpus = pd.read_csv('/kaggle/input/preprocessed-corpus/preprocessed_corpus.csv')

with open('/kaggle/input/predict-dissimilar/kaggle/working/predict_most_dissimilar.json', 'r') as f:
    predictions = json.load(f)

training_data = []

for pred in predictions:
    qid = pred['query_id']
    neg_cid = pred['candidates']['doc_ids'][0]  #Negative cid
    
    # Query text from train.csv
    query_row = df_train[df_train['qid'] == qid]
    if query_row.empty:
        continue
    query_text = query_row['question'].iloc[0]
    
    # Positive -> ground truth - given in train.csv
    ground_truth_str = query_row['cid'].iloc[0]
    ground_truth_cids = ground_truth_str.strip('[]').split()
    pos_cid = ground_truth_cids[0]  
    
    pos_matches = df_corpus.loc[df_corpus['cid'] == int(pos_cid), 'text']
    neg_matches = df_corpus.loc[df_corpus['cid'] == int(neg_cid), 'text']
    
    if pos_matches.empty or neg_matches.empty:
        continue  
    
    pos_text = pos_matches.iloc[0]
    neg_text = neg_matches.iloc[0]
    
    entry = {
        "query": query_text,
        "pos": [pos_text],
        "neg": [neg_text]
    }
    training_data.append(entry)

with open('fine_tune_training.json', 'w', encoding='utf-8') as f:
    for entry in training_data:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')



# Fine-tune bkai-foundation-models/vietnamese-bi-encoder
- Use the given fine_tune_training.json to fine-tune the model
- This part was originally done on a high-performance server with a RTX 4090, but for demonstrative purpose, we will utilize Kaggle for now.
- Since the waiting time would be rigorous, in this code we only run with 1 epoch, while the orignal version ran with 3.

In [1]:
from datasets import Dataset, load_dataset
import os
! pip install -U accelerate
! pip install -U transformers sentence_transformers
os.environ["WANDB_DISABLED"] = "true"
import json
from sentence_transformers import (
    SentenceTransformer,
    losses,
    InputExample
)
from torch.utils.data import DataLoader


'''
    #Input:
    Model: bkai-foundation-models/vietnamese-bi-encoder
    Negative pairing dataset: fine_tune_training.json

    #Output:
    Fine-tuned model.
'''

model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]
    
    train_examples = []
    for item in data:
        query = item['query']
        pos = item['pos'][0] 
        
        # Positive
        train_examples.append(InputExample(
            texts=[query, pos],
            label=1.0  
        ))
        
        # Negative
        for neg in item['neg']:
            train_examples.append(InputExample(
                texts=[query, neg],
                label=0.0 
            ))
    
    return train_examples

train_examples = load_json_data('/kaggle/input/negativepairing/fine_tune_training.json')

train_dataloader = DataLoader(
    train_examples,
    shuffle=True,
    batch_size=48
)

train_loss = losses.MultipleNegativesRankingLoss(model)

warmup_steps = int(len(train_dataloader) * 0.1)  

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1, #Kaggle: 1 epoch - HPC: 3 epochs
    warmup_steps=warmup_steps,
    optimizer_params={'lr': 2e-5},
    output_path='/kaggle/working/finetuned',
    save_best_model=True,
    show_progress_bar=True,
    checkpoint_path='/kaggle/working/checkpoints',
    checkpoint_save_steps=7500,  
    checkpoint_save_total_limit=1 
)

model.save('/kaggle/working/final') 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 9.2 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 79.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,3.084500
1000,1.987400
1500,1.683600
2000,1.501000
2500,1.370300
3000,1.264400
3500,1.206300
4000,1.132600
4500,1.122100


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [5]:
import os
import subprocess
from IPython.display import FileLink, display

#Save finetuned model(1 epoch version)
def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

# download_file('/kaggle/working', 'encoded_public_test_finetuned')
download_file('encoded_corpus_finetuned.json', 'encoded_corpus_finetuned')

/kaggle/working/encoded_corpus_finetuned.zip

# Encode with fine-tuned model
- preprocessed_corpus.csv to encoded_corpus_finetuned.json
- public_test.csv to encoded_public_test_finetuned.json

In [4]:
import pandas as pd
import numpy as np
import torch
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

'''
    Input:
        Fine-tuned model: finetuned-biencoder
        preprocessed_corpus.csv
    Output:
        encoded_corpus_finetuned.json
'''

# Encode corpus.csv
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_csv('/kaggle/input/preprocessed-corpus/preprocessed_corpus.csv')

#Not modified yet
model = SentenceTransformer('/kaggle/input/finetuned-biencoder/final')
model = model.to(device)

def encode(lst = [], convert_to_tensor=True, batch_size=128):
    vectors = []
    with tqdm(total = len(lst), desc="Encoding texts") as pbar:
        # Process in batches
        for i in range(0, len(lst), batch_size):
            batch = lst[i:i + batch_size]
            encoded_batch = model.encode(batch, convert_to_tensor=True, device=device)
            # Move to CPU before converting to numpy
            if torch.cuda.is_available():
                encoded_batch = encoded_batch.cpu()
            vectors.extend([np.array(arr) for arr in encoded_batch.numpy()])
            pbar.update(len(batch))
    return vectors

df['vector'] = encode(lst=list(df['text']))

df.to_json('encoded_corpus_finetuned.json')


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Encoding texts:   0%|          | 0/261597 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding texts:   0%|          | 128/261597 [00:00<29:17, 148.78it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Encoding texts: 100%|█████████▉| 261376/261597 [29:59<00:01, 141.94it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Encoding texts: 100%|█████████▉| 261504/261597 [30:00<00:00, 145.77it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Encoding texts: 100%|██████████| 261597/261597 [30:01<00:00, 145.22it/s]


In [ ]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

'''
    Input:
        Fine-tuned model
        public_test.csv
    Output:
        encoded_public_test_finetuned.json
'''

# Encode public_test
df = pd.read_csv('/kaggle/input/bkai-ai-track2-legal-document-retrieval/Legal Document Retrieval/public_test.csv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('/kaggle/input/finetuned-biencoder/final', device=device)

def encode(lst = [], convert_to_tensor=True, batch_size=128):
    vectors = []
    with tqdm(total=len(lst), desc="Encoding questions") as pbar:
        for i in range(0, len(lst), batch_size):
            batch = lst[i:i + batch_size]
            encoded_batch = model.encode(batch, convert_to_tensor=True)
            if device == 'cuda':
                encoded_batch = encoded_batch.cpu()
            vectors.extend([np.array(arr) for arr in encoded_batch.numpy()])
            pbar.update(len(batch))
    return vectors

df['question_vector'] = encode(lst=list(df['question']))

output_df = df[['question', 'qid', 'question_vector']]

output_df.to_json('encoded_public_test_finetuned.json')

# Predict the 50 top relevant responses for public_test using the fine-tuned model
- Input: encoded_corpus_finetuned.json and encoded_public_test.json

In [2]:
import torch
import torch.nn.functional as F
import pandas as pd
import zipfile
import json
'''
    Input: encoded_corpus_finetuned.json and encoded_public_test.json
    Output: top 50 predictions
'''

train_df = pd.read_json('/kaggle/input/finetuned-corpus-and-test/encoded_public_test_finetuned/kaggle/working/encoded_public_test_finetuned.json')
corpus_df = pd.read_json('/kaggle/input/finetuned-corpus-and-test/encoded_corpus_finetuned/kaggle/working/encoded_corpus.json')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


train_ids = train_df['qid'].tolist()
train_vectors = torch.tensor(train_df['vector'].tolist(), dtype=torch.float32).to(device)
corpus_ids = corpus_df['cid'].tolist()
try:
    corpus_vectors = torch.tensor(corpus_df['vector'].tolist(), dtype=torch.float32).to(device)
except:
    corpus_vectors = torch.tensor(corpus_df['question_vector'].tolist(), dtype=torch.float32).to(device)


train_vectors = F.normalize(train_vectors, p=2, dim=1)
corpus_vectors = F.normalize(corpus_vectors, p=2, dim=1)

similarity_matrix = torch.matmul(train_vectors, corpus_vectors.T)

top_k = 50  
top_k_values, top_k_indices = torch.topk(similarity_matrix, k=top_k, dim=1)

json_results = []

with open('predict_top50_finetuned.txt', 'w') as f:
    for i, (indices, scores) in enumerate(zip(top_k_indices, top_k_values)):
        qid = train_ids[i]
        
        top_cids = [str(corpus_ids[idx.item()]) for idx in indices]
        similarity_scores = [score.item() for score in scores]
        
        line = f"{qid} {' '.join(top_cids)}\n"
        f.write(line)
        
        json_entry = {
            "query_id": qid,
            "candidates": {
                "doc_ids": top_cids,
                "scores": similarity_scores,
            }
        }
        json_results.append(json_entry)

with open('predict_top50_finetuned.json', 'w', encoding='utf-8') as f:
    json.dump(json_results, f, ensure_ascii=False, indent=2)

with zipfile.ZipFile('predict_top50_finetuned.zip', 'w') as zipf:
    zipf.write('predict_top50_finetuned.txt')
    zipf.write('predict_top50_finetuned.json')

Using device: cuda


# Re-rank the top 50 prediction using cross-encoder

In [5]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import json

predictions = pd.read_json('/kaggle/input/top50finetuned/predict_top50_finetuned.json')
test_df = pd.read_csv('/kaggle/input/bkai-ai-track2-legal-document-retrieval/Legal Document Retrieval/public_test.csv')
corpus_df = pd.read_csv('/kaggle/input/preprocessed-corpus/preprocessed_corpus.csv')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained('namdp-ptit/ViRanker')
model = AutoModelForSequenceClassification.from_pretrained('namdp-ptit/ViRanker').to(device)
model.eval()

reranked_results = []

for _, row in predictions.iterrows():
    qid = row['query_id']  
    query_text = test_df['question'][test_df['qid'] == qid].iloc[0]
    doc_ids = row['candidates']['doc_ids']
    
    print(f"Processing qid: {qid}")
    print(f"First few doc_ids: {doc_ids[:3]}")
    
    pairs = []
    for doc_id in doc_ids:
        texts = corpus_df.loc[corpus_df['cid'] == doc_id, 'text'].tolist()
        for text in texts:
            pairs.append((query_text, text))

    batch_size = 8 
    all_scores = []

    with torch.no_grad():
        for i in range(0, len(pairs), batch_size):
            batch_pairs = pairs[i:i + batch_size]
            inputs = tokenizer(batch_pairs, padding=True, truncation=True, 
                            return_tensors='pt', max_length=512)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            scores = model(**inputs, return_dict=True).logits.view(-1,).float()
            all_scores.extend(scores.cpu().numpy().tolist())

    json_entry = {
        "query_id": qid,
        "candidates": {
            "doc_ids": doc_ids,
            "scores": all_scores
        }
    }
    reranked_results.append(json_entry)

with open('predict_top50_reranked.json', 'w') as f:
    json.dump(reranked_results, f, indent=2)


Processing qid: 98440
First few doc_ids: ['77191', '230002', '111380']
Processing qid: 105737
First few doc_ids: ['51573', '229317', '470960']
Processing qid: 106239
First few doc_ids: ['214239', '101722', '184177']
Processing qid: 79491
First few doc_ids: ['50701', '64708', '64709']
Processing qid: 130557
First few doc_ids: ['36248', '3788', '193882']
Processing qid: 109476
First few doc_ids: ['139187', '557644', '572543']
Processing qid: 64585
First few doc_ids: ['136490', '35028', '256964']
Processing qid: 5785
First few doc_ids: ['108373', '63709', '65892']
Processing qid: 159576
First few doc_ids: ['141320', '126678', '162297']
Processing qid: 36563
First few doc_ids: ['534271', '214685', '593864']
Processing qid: 43191
First few doc_ids: ['71162', '167857', '71161']
Processing qid: 139105
First few doc_ids: ['25388', '73944', '25362']
Processing qid: 102773
First few doc_ids: ['88260', '72922', '210050']
Processing qid: 32598
First few doc_ids: ['90380', '140526', '236329']
Proce